In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime 

%matplotlib inline
pd.set_option('display.max_rows', 500)

![](Business_understanding.jpeg)

# Data Preparation

- Cleaning and transforming raw data before processing and analysis. 
- Undertand the final data structure with the help of visual analytics. 

## John Hopkins (GITHUB) Data

In [ ]:
data_raw = pd.read_csv("E:/ads_covid-19/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")

In [ ]:
data_raw.head()

In [ ]:
date = data_raw.columns[4:]
df_plot = pd.DataFrame({'Date': date})

In [ ]:
df_plot.shape

In [ ]:
data_raw[data_raw['Country/Region'] == 'US'].iloc[:,4::].shape

In [ ]:
country_list = ['India', 'US', 'Italy', 'Spain', 'Germany'] # Small country list to start with.

In [ ]:
# Slice the data for selected countries. 
for each in country_list:
    df_plot[each] = np.array(data_raw[data_raw['Country/Region'] == each].iloc[:,4::].sum(axis=0)).T
df_plot.head()

In [ ]:
df_plot.set_index('Date')

In [ ]:
df_plot.plot()

## Data Type 'Date'

In [ ]:
df_plot.head()

In [ ]:
time_idx = [datetime.strptime(each, '%m/%d/%y') for each in df_plot.Date] # Convert to datetime

In [ ]:
time_str = [datetime.strftime(each, "%Y-%m-%d") for each in time_idx] # Convert back to str

In [ ]:
df_plot["Date"] = time_str

In [ ]:
df_plot.to_csv("E:/ads_covid-19/data/processed/COVID_flat_small_table.csv", sep = ';', index = None)

# Relational Data Model- Defining a Primary Key

- In the relational model of databases, a primary key is a specific choice of a minimal set of attributes(columns) that uniquely specify a tuple(row) in a relation(table). (source: https://en.wikipedia.org/wiki/Primary_key)



- A primary key’s main features are:

    1) It must contain a unique value for each row of data.
      
    2) It cannot contain null values.    
    
    3) Every row must have a primary key value.  
    source: https://www.techopedia.com/definition/5547/primary-key

In [ ]:
data_raw = pd.read_csv("E:/ads_covid-19/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")

In [ ]:
data_raw.drop(["Lat", "Long"], axis=1, inplace = True)

In [ ]:
data_raw.rename(columns={'Province/State':"state", 'Country/Region' : 'country'}, inplace = True)
data_raw['state'] = data_raw['state'].fillna('no')
data_raw.head()

In [ ]:
pd_data_base = data_raw.set_index(['state', 'country']).T
pd_data_base.head()

In [ ]:
pd_relational_model = pd_data_base.stack(level=[0,1]).reset_index()
pd_relational_model.head()

In [ ]:
pd_relational_model.rename(columns={'level_0':'date', 0:'confirmed'}, inplace = True)

In [ ]:
pd_relational_model.head()

In [ ]:
pd_relational_model['date'] = pd_relational_model['date'].astype('datetime64[ns]')  # convert to datetime
pd_relational_model['confirmed'] = pd_relational_model['confirmed'].astype(int)    # convert to integer

In [ ]:
pd_relational_model.dtypes

In [ ]:
pd_relational_model.to_csv('E:/ads_covid-19/data/processed/COVID_relational_confirmed.csv', sep = ';', index = None)

In [ ]:
print(pd_relational_model[pd_relational_model['country']=='US'].tail()) 
# time_series_covid19_confirmed_global.csv does not contain individual states data of US.
# Individual states data of US is stored in different csv file.


## US States Data: John Hopkins Data Set

In [ ]:
pd_raw_US = pd.read_csv('E:/ads_covid-19/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')

In [ ]:
pd_raw_US.head()

In [ ]:
# Remove all non-relevant data
pd_raw_US.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Country_Region', 'Lat', 'Long_', 'Combined_Key'], axis=1, inplace=True)
pd_data_base_US = pd_raw_US.rename(columns={'Province_State':'state'}).copy()

In [ ]:
pd_relational_model_US = pd_data_base_US.set_index(['state']) \
                                        .T \
                                        .stack() \
                                        .reset_index() \
                                        .rename(columns = {'level_0':'date', 0:'confirmed'})
pd_relational_model_US['country'] = 'US'
pd_relational_model_US.date = pd_relational_model_US.date.astype('datetime64[ns]')  ## convert to datetime

In [ ]:
pd_relational_model_US.head()

In [ ]:
# pd_relational_model_US is concatenated with pd_relational_model to add states data of US.

In [ ]:
pd_relational_model_all = pd_relational_model[pd_relational_model['country']!='US'].reset_index(drop=True)
pd_relational_model_all=pd.concat([pd_relational_model_all,pd_relational_model_US],ignore_index=True)

In [ ]:
pd_relational_model_all.to_csv('E:/ads_covid-19/data/processed/20200424_COVID_relational_confirmed.csv',sep=';',index=False)

### Cross check the US data

In [ ]:
test_US = pd_relational_model_all[pd_relational_model_all['country']=='US']

In [ ]:
test_US[['date', 'country', 'confirmed']].groupby(['date', 'country']).sum()

# RKI data

- Numbers are in German standard, i.e. "," indicates decimal and "." indicates thousands. 
- It must be converted to standard system.

In [ ]:
pd_RKI_raw = pd.read_csv('E:/ads_covid-19/data/raw/RKI/RKI_data.csv',sep=';')

In [ ]:
pd_RKI_raw.head()

In [ ]:
pd_RKI_raw.dtypes 

In [ ]:
pd_RKI = pd.read_csv('E:/ads_covid-19/data/raw/RKI/RKI_data.csv',sep=';', thousands = '.', decimal = ',')

In [ ]:
pd_RKI.head()

In [ ]:
pd_RKI.dtypes

In [ ]:
pd_RKI.to_csv('E:/ads_covid-19/data/processed/RKI_data_prepared.csv',sep=';', index = False) 